<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_30WMA_RSI14_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2024-05-11 14:12:16


In [32]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [33]:
import pandas as pd
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
#mypf = mypf[~mypf['Criteria'].isin(['V40','V40N'])]
mypf = mypf[mypf['TFrame'] == 'W']

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ','ADANITRANS', 'EQUITAS']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

all_prospects = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

all_prospects.sort()
#stock_ids

In [34]:
# import necessary libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [35]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [36]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 10
      #mid_window = 50
      long_window = 30
      moving_avg = 'WMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1wk')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['30_WMA'] = round(stock_df['Close'].ewm(span=long_window, adjust=False).mean(),0)
      stock_df['Dev%_30'] = round((stock_df['Close'] - stock_df['30_WMA'])*100/stock_df['30_WMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df['ATH%'] = round((stock_df['Max'] - stock_df['Close'])*100/stock_df['Close'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      #df_prec_dev = df_prec_dev.append(tmp)
      df_prec_dev = pd.concat([df_prec_dev, tmp])
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()


def get_stock_buy_sell(stock_symbol):
    #date = (datetime.date.today() + datetime.timedelta(days=0))
    df_prec_dev = stock_prec_dev(stock_symbol)
    df_prec_dev['Signal'] = 0.0
    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] > df_prec_dev['30_WMA']) & (df_prec_dev['RSI_14'] > 60) ), 1.0, df_prec_dev['Signal'])

    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] < df_prec_dev['30_WMA']) & (df_prec_dev['RSI_14'] < 60) ), -1.0, df_prec_dev['Signal'])

    df_prec_dev['Position'] = df_prec_dev['Signal'].diff()

    df_pos = df_prec_dev[(df_prec_dev['Position'] >= 1) | (df_prec_dev['Position'] <= -1)]
    df_pos = df_pos[df_pos['Signal'] != 0]
    df_pos['Prev_Signal'] = df_pos['Signal'].shift(1)
    df_pos = df_pos[df_pos['Signal'] != df_pos['Prev_Signal']]
    df_pos.drop('Prev_Signal', axis=1, inplace=True)

    return df_pos.tail(1)

def get_weekly_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        #df_buy_sell = df_buy_sell.append(df_tmp)
        df_buy_sell = pd.concat([df_buy_sell, df_tmp])


    df_buy_sell['Position'] = df_buy_sell['Position'].apply(lambda x: 'Buy' if x >= 1 else 'Sell')
    return df_buy_sell.sort_index()


In [37]:
df_weekly_buy_sell = get_weekly_buy_sell()
df_weekly_buy_sell.index = df_weekly_buy_sell.index.date
df_weekly_buy_sell

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,ATH%,RSI_14,Signal,Position,Criteria
2023-08-21,8.0,6.0,26.67,JPPOWER.NS,23.25,5.60,35.71,205.92,75.0,1.0,Buy,AVSP
2023-08-21,1023.0,807.0,26.78,CAPLIPOINT.NS,1572.35,696.62,46.86,53.69,83.0,1.0,Buy,AV40N
2023-08-21,138.0,119.0,16.28,CASTROLIND.NS,223.45,105.42,31.26,61.49,85.0,1.0,Buy,AVR
2023-08-21,4857.0,3957.0,22.74,DIXON.NS,8580.15,3033.47,60.11,76.66,77.0,1.0,Buy,AV40N
2023-08-21,818.0,831.0,-1.59,SBICARD.NS,1128.43,680.09,20.25,37.99,32.0,-1.0,Sell,AVSP
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-29,325.0,296.0,9.95,CROMPTON.NS,483.75,251.86,29.22,48.64,64.0,1.0,Buy,AAR
2024-05-06,3271.0,3371.0,-2.95,LT.NS,3763.90,2185.12,49.71,15.05,45.0,-1.0,Sell,AV200
2024-05-06,523.0,473.0,10.53,EMAMILTD.NS,575.65,376.45,38.88,10.11,62.0,1.0,Buy,AV200
2024-05-06,2278.0,2432.0,-6.33,SRF.NS,2730.18,2163.09,5.31,19.85,41.0,-1.0,Sell,AVR


In [38]:
df_weekly_buy_sell[df_weekly_buy_sell['Position'] == 'Sell'].tail(20)

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,ATH%,RSI_14,Signal,Position,Criteria
2024-03-11,510.0,517.0,-1.34,5PAISA.NS,692.60,313.45,62.72,35.79,48.0,-1.0,Sell,AV40N
2024-03-11,224.0,246.0,-8.88,RBLBANK.NS,673.82,143.01,56.74,200.61,41.0,-1.0,Sell,AVSP
2024-03-11,51.0,53.0,-4.25,DEN.NS,97.10,30.35,67.22,91.33,45.0,-1.0,Sell,AAR
2024-03-11,53.0,54.0,-1.76,COFFEEDAY.NS,249.60,33.25,59.55,370.50,48.0,-1.0,Sell,AVR
2024-03-11,373.0,418.0,-10.67,VAIBHAVGBL.NS,953.31,291.71,28.00,155.31,40.0,-1.0,Sell,AVR
2024-03-11,200.0,212.0,-5.64,PARAGMILK.NS,266.30,101.40,97.29,33.12,45.0,-1.0,Sell,AAR
2024-03-11,4928.0,5082.0,-3.03,ASTRAZEN.NS,6475.40,3187.81,54.59,31.40,45.0,-1.0,Sell,AV40N
2024-03-11,20.0,21.0,-3.10,HATHWAY.NS,48.95,13.25,53.58,140.54,46.0,-1.0,Sell,AVR
2024-03-11,170.0,171.0,-0.76,IBULHSGFIN.NS,631.88,97.33,74.36,272.35,49.0,-1.0,Sell,AVR
2024-03-18,4865.0,4876.0,-0.23,BRITANNIA.NS,5338.45,4484.35,8.49,9.73,48.0,-1.0,Sell,AAR


In [39]:
df_weekly_buy_sell[df_weekly_buy_sell['Position'] == 'Buy'].tail(20)

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,ATH%,RSI_14,Signal,Position,Criteria
2023-11-20,675.0,628.0,7.49,LICI.NS,1076.70,560.14,20.51,59.50,67.0,1.0,Buy,AV200
2023-12-04,97.0,88.0,10.61,SAIL.NS,168.55,81.04,20.11,73.16,65.0,1.0,Buy,AVR
2023-12-04,494.0,433.0,14.17,AMBUJACEM.NS,632.35,400.81,23.34,27.92,70.0,1.0,Buy,AVSP
2023-12-04,614.0,575.0,6.81,SBIN.NS,831.45,554.60,10.74,35.38,63.0,1.0,Buy,AVR
2023-12-04,750.0,672.0,11.54,IRCTC.NS,1074.40,606.23,23.64,43.34,66.0,1.0,Buy,AVSP
2024-01-01,450.0,409.0,10.11,AVANTIFEED.NS,687.57,360.97,24.76,52.67,68.0,1.0,Buy,AV200
2024-01-22,813.0,703.0,15.66,ARE&M.NS,1128.10,616.28,31.94,38.74,78.0,1.0,Buy,AV200
2024-02-12,1786.0,1557.0,14.70,METROPOLIS.NS,3398.33,1221.45,46.21,90.29,64.0,1.0,Buy,AVR
2024-03-18,4298.0,3851.0,11.60,DMART.NS,5323.75,3395.25,26.58,23.88,70.0,1.0,Buy,AAR
2024-04-01,952.0,893.0,6.61,SYMPHONY.NS,1459.96,828.30,14.93,53.36,61.0,1.0,Buy,AV40N


In [40]:
#tmp_df = df_daily_buy_sell.reset_index()
#tmp_df.to_csv('/content/drive/My Drive/data/stocks/myProspects-Buy-Sell-Signals.csv', index=False)

In [41]:
#stock_symbol = 'INFY.NS'
#plot_stock_buy_sell(stock_symbol)